<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/otto_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jsonlines
import jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Install Polars with all optional dependencies.
!pip install 'polars[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 MB 20.6 MB/s eta 0:00:00


In [ ]:
#importing Libraries
import polars as pl
from io import StringIO
import pandas as pd
import json
import os
import numpy as np
import csv
from datetime import datetime
import time
from datetime import timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/OTTO/train1.zip"

In [ ]:
train = pd.read_csv('train1.csv')

In [ ]:
train.head()

,session,aid,ts,type
0,0,1517085,1659304800025,clicks
1,0,1563459,1659304904511,clicks
2,0,1309446,1659367439426,clicks
3,0,16246,1659367719997,clicks
4,0,1781822,1659367871344,clicks


In [ ]:
train.session.nunique()

1000000

In [ ]:
train.type.nunique()

3

In [ ]:
train.shape

(39889600, 4)

In [ ]:
train[["session", "ts"]].groupby("session").head()

,session,ts
0,0,1659304800025
1,0,1659304904511
2,0,1659367439426
3,0,1659367719997
4,0,1659367871344
...,...,...
39889590,999999,1659378927661
39889591,999999,1659378968391
39889592,999999,1659378974466
39889593,999999,1659464364560


In [ ]:
# Time of session
time_elapsed = example_session[-1]["ts"] - example_session[0]["ts"]
# The timestamp is in milliseconds since 00:00:00 UTC on 1 January 1970
print(f'The first session elapsed: {str(timedelta(milliseconds=time_elapsed))} \n')

In [ ]:
%%time
train_df = train
train_df["minutes"] = train[["session", "ts"]].groupby("session").diff(-1)
# train_df = train_df.sort_values(['minutes'],ascending=False)    # 다음 행동까지 걸린 시간 내림차순 정렬


CPU times: user 4min 31s, sys: 16.4 s, total: 4min 47s
Wall time: 4min 40s


,session,aid,ts,type,minutes
0,0,1517085,1659304800025,clicks,-104486.0
1,0,1563459,1659304904511,clicks,-62534915.0
2,0,1309446,1659367439426,clicks,-280571.0
3,0,16246,1659367719997,clicks,-151347.0
4,0,1781822,1659367871344,clicks,-14452.0


In [ ]:
train_df.minutes*(-1/1000/60)

0               1.741433
1            1042.248583
2               4.676183
3               2.522450
4               0.240867
                ...     
39889595        0.567350
39889596        0.885833
39889597        5.888133
39889598    24457.408350
39889599             NaN
Name: minutes, Length: 39889600, dtype: float64

In [ ]:
train_action_df = train_df.copy()
train_action_df.aid = ' ' + train_df.aid.astype('str')
train_action_df = train_action_df.groupby(['session','type'])['aid'].sum().reset_index()

# session type 으로 묶어서 aid 모아보기

In [ ]:
# Let's look at an example session and print out some basic info

# Sample the first session in the df
example_session = train.iloc[0].item()
print(f'This session was {len(example_session)} actions long \n')
print(f'The first action in the session: \n {example_session[0]} \n')

# Time of session
time_elapsed = example_session[-1]["ts"] - example_session[0]["ts"]
# The timestamp is in milliseconds since 00:00:00 UTC on 1 January 1970
print(f'The first session elapsed: {str(timedelta(milliseconds=time_elapsed))} \n')

# Count the frequency of actions within the session
action_counts = {}
for action in example_session:
    action_counts[action['type']] = action_counts.get(action['type'], 0) + 1  
print(f'The first session contains the following frequency of actions: {action_counts}')

ValueError: ignored

In [ ]:
train.iloc[0]

session                0
aid              1517085
ts         1659304800025
type              clicks
Name: 0, dtype: object